# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
import gc
import os
import re
from typing import List, Dict, Tuple

import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as cs

from sklearn.cluster import KMeans, DBSCAN, MiniBatchKMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder

# import warnings
# warnings.filterwarnings('ignore')

In [2]:
# load files
# train = pl.read_parquet('/kaggle/input/aeroclub-recsys-2025/train.parquet')
# train_y = train['selected']
# train = train.drop('selected')

# data = pl.concat([train, pl.read_parquet('/kaggle/input/aeroclub-recsys-2025/test.parquet')])
data = pl.read_parquet('/kaggle/input/aeroclub-recsys-2025/train.parquet').drop('selected')

# del train
gc.collect()
# sample_submission = pl.read_parquet('/kaggle/input/aeroclub-recsys-2025/sample_submission.parquet')

0

In [3]:
data.head()

Id,bySelf,companyID,corporateTariffCode,frequentFlyer,nationality,isAccess3D,isVip,legs0_arrivalAt,legs0_departureAt,legs0_duration,legs0_segments0_aircraft_code,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_departureFrom_airport_iata,legs0_segments0_duration,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_seatsAvailable,legs0_segments1_aircraft_code,legs0_segments1_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_iata,legs0_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs0_segments1_cabinClass,legs0_segments1_departureFrom_airport_iata,legs0_segments1_duration,legs0_segments1_flightNumber,legs0_segments1_marketingCarrier_code,legs0_segments1_operatingCarrier_code,legs0_segments1_seatsAvailable,legs0_segments2_aircraft_code,legs0_segments2_arrivalTo_airport_city_iata,…,legs1_segments2_baggageAllowance_quantity,legs1_segments2_baggageAllowance_weightMeasurementType,legs1_segments2_cabinClass,legs1_segments2_departureFrom_airport_iata,legs1_segments2_duration,legs1_segments2_flightNumber,legs1_segments2_marketingCarrier_code,legs1_segments2_operatingCarrier_code,legs1_segments2_seatsAvailable,legs1_segments3_aircraft_code,legs1_segments3_arrivalTo_airport_city_iata,legs1_segments3_arrivalTo_airport_iata,legs1_segments3_baggageAllowance_quantity,legs1_segments3_baggageAllowance_weightMeasurementType,legs1_segments3_cabinClass,legs1_segments3_departureFrom_airport_iata,legs1_segments3_duration,legs1_segments3_flightNumber,legs1_segments3_marketingCarrier_code,legs1_segments3_operatingCarrier_code,legs1_segments3_seatsAvailable,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,profileId,ranker_id,requestDate,searchRoute,sex,taxes,totalPrice,__index_level_0__
i64,bool,i64,i64,str,i64,bool,bool,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,…,f64,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,str,datetime[ns],str,bool,f64,f64,i64
0,true,57323,null,"""S7/SU/UT""",36,false,false,"""2024-06-15T16:20:00""","""2024-06-15T15:40:00""","""02:40:00""","""YK2""","""KJA""","""KJA""",1.0,0.0,1.0,"""TLK""","""02:40:00""","""216""","""KV""","""KV""",9.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,1,2087645,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",true,370.0,16884.0,0
1,true,57323,123,"""S7/SU/UT""",36,true,false,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""","""07:25:00""","""E70""","""OVB""","""OVB""",1.0,0.0,1.0,"""TLK""","""02:50:00""","""5358""","""S7""","""S7""",4.0,"""E70""","""KJA""","""KJA""",1.0,0.0,1.0,"""OVB""","""01:20:00""","""5311""","""S7""","""S7""",4.0,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2300.0,null,1.0,3500.0,null,1.0,1.0,1,2087645,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",true,2240.0,51125.0,1
2,true,57323,null,"""S7/SU/UT""",36,false,false,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""","""07:25:00""","""E70""","""OVB""","""OVB""",1.0,0.0,1.0,"""TLK""","""02:50:00""","""5358""","""S7""","""S7""",4.0,"""E70""","""KJA""","""KJA""",1.0,0.0,1.0,"""OVB""","""01:20:00""","""5311""","""S7""","""S7""",4.0,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2300.0,null,1.0,3

In [9]:
data.filter(pl.col('isVip') == 1).limit(1000)

Id,bySelf,companyID,corporateTariffCode,frequentFlyer,nationality,isAccess3D,isVip,legs0_arrivalAt,legs0_departureAt,legs0_duration,legs0_segments0_aircraft_code,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_departureFrom_airport_iata,legs0_segments0_duration,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_seatsAvailable,legs0_segments1_aircraft_code,legs0_segments1_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_iata,legs0_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs0_segments1_cabinClass,legs0_segments1_departureFrom_airport_iata,legs0_segments1_duration,legs0_segments1_flightNumber,legs0_segments1_marketingCarrier_code,legs0_segments1_operatingCarrier_code,legs0_segments1_seatsAvailable,legs0_segments2_aircraft_code,legs0_segments2_arrivalTo_airport_city_iata,…,legs1_segments2_baggageAllowance_quantity,legs1_segments2_baggageAllowance_weightMeasurementType,legs1_segments2_cabinClass,legs1_segments2_departureFrom_airport_iata,legs1_segments2_duration,legs1_segments2_flightNumber,legs1_segments2_marketingCarrier_code,legs1_segments2_operatingCarrier_code,legs1_segments2_seatsAvailable,legs1_segments3_aircraft_code,legs1_segments3_arrivalTo_airport_city_iata,legs1_segments3_arrivalTo_airport_iata,legs1_segments3_baggageAllowance_quantity,legs1_segments3_baggageAllowance_weightMeasurementType,legs1_segments3_cabinClass,legs1_segments3_departureFrom_airport_iata,legs1_segments3_duration,legs1_segments3_flightNumber,legs1_segments3_marketingCarrier_code,legs1_segments3_operatingCarrier_code,legs1_segments3_seatsAvailable,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,profileId,ranker_id,requestDate,searchRoute,sex,taxes,totalPrice,__index_level_0__
i64,bool,i64,i64,str,i64,bool,bool,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,…,f64,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,str,datetime[ns],str,bool,f64,f64,i64
6046,true,43509,null,null,6,false,true,"""2024-06-18T18:35:00""","""2024-06-18T14:50:00""","""03:45:00""","""333""","""AER""","""AER""",0.0,0.0,1.0,"""SVO""","""03:45:00""","""11""","""N4""","""N4""",9.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,1,1708049,"""b728143078f0455f856d14e4388d93…",2024-05-17 09:12:57,"""MOWAER""",true,643.0,11143.0,6046
6047,true,43509,null,null,6,false,true,"""2024-06-18T18:35:00""","""2024-06-18T14:50:00""","""03:45:00""","""333""","""AER""","""AER""",1.0,0.0,1.0,"""SVO""","""03:45:00""","""11""","""N4""","""N4""",9.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,1,1708049,"""b728143078f0455f856d14e4388d93…",2024-05-17 09:12:57,"""MOWAER""",true,643.0,13243.0,6047
6048,true,43509,null,null,6,false,true,"""2024-06-18T18:35:00""","""2024-06-18T14:50:00""","""03:45:00""","""333""","""AER""","""AER""",1.0,0.0,1.0,"""SVO""","""03:45:00""","""11""","""N4""","""N4""",9.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,1,1708049,"""b728143078f0455f856d14e4388d93…",2024-05-17 09:12:57,"""MOWAER""",true,643.0,16143.0,6048
6

In [12]:
data.filter((pl.col('profileId').is_in([2793673, 118809]) & (pl.col('isVip') == 0)))

Id,bySelf,companyID,corporateTariffCode,frequentFlyer,nationality,isAccess3D,isVip,legs0_arrivalAt,legs0_departureAt,legs0_duration,legs0_segments0_aircraft_code,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_departureFrom_airport_iata,legs0_segments0_duration,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_seatsAvailable,legs0_segments1_aircraft_code,legs0_segments1_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_iata,legs0_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs0_segments1_cabinClass,legs0_segments1_departureFrom_airport_iata,legs0_segments1_duration,legs0_segments1_flightNumber,legs0_segments1_marketingCarrier_code,legs0_segments1_operatingCarrier_code,legs0_segments1_seatsAvailable,legs0_segments2_aircraft_code,legs0_segments2_arrivalTo_airport_city_iata,…,legs1_segments2_baggageAllowance_quantity,legs1_segments2_baggageAllowance_weightMeasurementType,legs1_segments2_cabinClass,legs1_segments2_departureFrom_airport_iata,legs1_segments2_duration,legs1_segments2_flightNumber,legs1_segments2_marketingCarrier_code,legs1_segments2_operatingCarrier_code,legs1_segments2_seatsAvailable,legs1_segments3_aircraft_code,legs1_segments3_arrivalTo_airport_city_iata,legs1_segments3_arrivalTo_airport_iata,legs1_segments3_baggageAllowance_quantity,legs1_segments3_baggageAllowance_weightMeasurementType,legs1_segments3_cabinClass,legs1_segments3_departureFrom_airport_iata,legs1_segments3_duration,legs1_segments3_flightNumber,legs1_segments3_marketingCarrier_code,legs1_segments3_operatingCarrier_code,legs1_segments3_seatsAvailable,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,profileId,ranker_id,requestDate,searchRoute,sex,taxes,totalPrice,__index_level_0__
i64,bool,i64,i64,str,i64,bool,bool,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,…,f64,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,str,datetime[ns],str,bool,f64,f64,i64


In [4]:
data.tail()

Id,bySelf,companyID,corporateTariffCode,frequentFlyer,nationality,isAccess3D,isVip,legs0_arrivalAt,legs0_departureAt,legs0_duration,legs0_segments0_aircraft_code,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_departureFrom_airport_iata,legs0_segments0_duration,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_seatsAvailable,legs0_segments1_aircraft_code,legs0_segments1_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_iata,legs0_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs0_segments1_cabinClass,legs0_segments1_departureFrom_airport_iata,legs0_segments1_duration,legs0_segments1_flightNumber,legs0_segments1_marketingCarrier_code,legs0_segments1_operatingCarrier_code,legs0_segments1_seatsAvailable,legs0_segments2_aircraft_code,legs0_segments2_arrivalTo_airport_city_iata,…,legs1_segments2_baggageAllowance_quantity,legs1_segments2_baggageAllowance_weightMeasurementType,legs1_segments2_cabinClass,legs1_segments2_departureFrom_airport_iata,legs1_segments2_duration,legs1_segments2_flightNumber,legs1_segments2_marketingCarrier_code,legs1_segments2_operatingCarrier_code,legs1_segments2_seatsAvailable,legs1_segments3_aircraft_code,legs1_segments3_arrivalTo_airport_city_iata,legs1_segments3_arrivalTo_airport_iata,legs1_segments3_baggageAllowance_quantity,legs1_segments3_baggageAllowance_weightMeasurementType,legs1_segments3_cabinClass,legs1_segments3_departureFrom_airport_iata,legs1_segments3_duration,legs1_segments3_flightNumber,legs1_segments3_marketingCarrier_code,legs1_segments3_operatingCarrier_code,legs1_segments3_seatsAvailable,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,profileId,ranker_id,requestDate,searchRoute,sex,taxes,totalPrice,__index_level_0__
i64,bool,i64,i64,str,i64,bool,bool,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,…,f64,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,str,datetime[ns],str,bool,f64,f64,i64
18146427,true,54154,44,null,36,true,false,"""2024-11-06T09:45:00""","""2024-11-05T20:50:00""","""10:55:00""","""738""","""SGC""","""SGC""",0.0,0.0,1.0,"""VKO""","""03:20:00""","""247""","""UT""","""UT""",3.0,"""AT7""","""SVX""","""SVX""",0.0,0.0,1.0,"""SGC""","""02:30:00""","""111""","""UT""","""UT""",5.0,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,24000.0,null,1.0,0.0,null,0.0,0.0,1,3046852,"""88f8c53a28bf4f438941fd67338009…",2024-10-29 12:46:20,"""MOWSVX/SVXMOW""",true,5560.0,30730.0,18146427
18146428,true,54154,44,null,36,true,false,"""2024-11-05T20:00:00""","""2024-11-05T00:20:00""","""17:40:00""","""738""","""UFA""","""UFA""",0.0,0.0,1.0,"""VKO""","""02:05:00""","""363""","""UT""","""UT""",3.0,"""AT7""","""SVX""","""SVX""",0.0,0.0,1.0,"""UFA""","""01:30:00""","""106""","""UT""","""UT""",9.0,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,24000.0,null,1.0,0.0,null,0.0,0.0,1,3046852,"""88f8c53a28bf4f438941fd67338009…",2024-10-29 12:46:20,"""MOWSVX/SVXMOW""",true,5560.0,27660.0,18146428
18146429,true,54154,44,null,36,true,false,"""2024-11-05T20:00:00""","""2024-11-05T00:20:00""","""17:40:00""","""738""","""UFA""","""UFA""",0.0,0.0,1.0,"""VKO""","""02:05:00""","""363""","""UT""","""UT""",3.0,"""AT7""","""SVX""","""SVX""",0.0,0.0,1.0,"""UFA""","""01:30:00""","""106""","""UT""","""UT""",9.0,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,

In [5]:
data.shape

(18145372, 126)

In [5]:
# determine the number of null values for each feature
results = []
for col in data.columns:
    null_count = data.select(pl.col(col).is_null().sum()).item()
    results.append({
        'column': col,
        'data_type': str(data.schema[col]),
        'null_count': null_count,
        'null_percentage': round(null_count / len(data) * 100, 2)
    })
null_summary = pl.DataFrame(results)

In [6]:
# features with no NULL values:
null_summary.filter(pl.col('null_count') == 0)

column,data_type,null_count,null_percentage
str,str,i64,f64
"""Id""","""Int64""",0,0.0
"""bySelf""","""Boolean""",0,0.0
"""companyID""","""Int64""",0,0.0
"""nationality""","""Int64""",0,0.0
"""isAccess3D""","""Boolean""",0,0.0
…,…,…,…
"""searchRoute""","""String""",0,0.0
"""sex""","""Boolean""",0,0.0
"""taxes""","""Float64""",0,0.0


In [7]:
# features with NULL values < 10%
null_summary.filter((pl.col("null_count") > 0) & (pl.col("null_percentage") < 10.0)).sort("null_count")

column,data_type,null_count,null_percentage
str,str,i64,f64
"""legs0_segments0_departureFrom_…","""String""",2,0.0
"""legs0_segments0_arrivalTo_airp…","""String""",6,0.0
"""legs0_segments0_aircraft_code""","""String""",14,0.0
"""legs0_segments0_arrivalTo_airp…","""String""",113,0.0
"""legs0_segments0_baggageAllowan…","""Float64""",1064,0.01
…,…,…,…
"""pricingInfo_isAccessTP""","""Float64""",905045,4.99
"""miniRules0_monetaryAmount""","""Float64""",1395743,7.69
"""miniRules1_monetaryAmount""","""Float64""",1395743,7.69


In [8]:
# features with NULL values 10% - 50%
null_summary.filter((pl.col("null_percentage") >= 10.0) & (pl.col("null_percentage") < 50.0)).sort("null_count")

column,data_type,null_count,null_percentage
str,str,i64,f64
"""legs1_arrivalAt""","""String""",4387201,24.18
"""legs1_departureAt""","""String""",4387201,24.18
"""legs1_duration""","""String""",4387201,24.18
"""legs1_segments0_aircraft_code""","""String""",4387201,24.18
"""legs1_segments0_arrivalTo_airp…","""String""",4387201,24.18
…,…,…,…
"""legs1_segments0_arrivalTo_airp…","""String""",4387237,24.18
"""legs1_segments0_cabinClass""","""Float64""",4525013,24.94
"""legs1_segments0_baggageAllowan…","""Float64""",4527297,24.95


At first glance, I would suspect these nulls indicate one-way tickets. Might need to figure out how to handle return flight selection.

In [9]:
# features with NULL values > 50%
null_summary.filter((pl.col("null_percentage") >= 50.0)).sort("null_count")

column,data_type,null_count,null_percentage
str,str,i64,f64
"""corporateTariffCode""","""Int64""",9233925,50.89
"""frequentFlyer""","""String""",12012727,66.2
"""legs0_segments1_arrivalTo_airp…","""String""",14311665,78.87
"""legs0_segments1_departureFrom_…","""String""",14311665,78.87
"""legs0_segments1_duration""","""String""",14311665,78.87
…,…,…,…
"""legs1_segments3_duration""","""String""",18145366,100.0
"""legs1_segments3_flightNumber""","""String""",18145366,100.0
"""legs1_segments3_marketingCarri…","""String""",18145366,100.0


These features are missing a large percentage of values. Given the description provided, we will do the following:
* corporateTariffCode (Corporate tariff code for businesss travel policies): impute to 0 (zero)
* frequentFlyer (Frequent flyer program status): impute to 'NONE'
* miniRules0_percentage (Percentage penalty for cancellation): impute to 0.0 (zero)
* miniRules1_percentage (Percentage penalty for exchange): impute to 0.0 (zero)
* legs[0,1]_segments[1-3]_* (flight segments): evaluate whether to ignore all but segment 1 or impute from segment 1

In [5]:
# check how many unique search sessions there are
print(f"unique sessions (ranker_id): {data['ranker_id'].unique().count()}")

unique sessions (ranker_id): 105539


In [6]:
# check how many unique customers
print(f"unique customers: {data['profileId'].unique().count()}")

unique customers: 32922


In [7]:
# let's look at carrier codes
unique_carrier_codes = (
    data.select(cs.contains("Carrier_code"))
    .unpivot()
    .select("value")
    .filter(pl.col("value").is_not_null())
    .unique().to_series().to_list()
)
print(unique_carrier_codes)

['WK', 'BL', '2C', 'UL', '7C', 'EN', 'EW', 'NI', 'AD', 'UX', 'J2', 'KN', 'VZ', 'T7', '4R', 'СИ', 'LJ', 'AI', 'YK', 'S7', 'AT', 'AH', 'OO', 'OS', 'PI', '9F', 'OK', 'AM', 'W2', 'LX', 'OU', 'BA', 'SU', 'ME', 'R6', 'UO', 'OD', 'I2', 'DL', 'TU', 'OM', 'KB', 'CM', 'AA', 'DP', 'SQ', 'UK', 'NP', 'OZ', 'UT', 'SZ', '4Z', 'ИК', 'AY', '0V', 'ТГ', 'SK', 'OV', 'KQ', 'A5', 'I5', 'DV', 'GS', 'CZ', 'JM', 'JD', 'R3', 'QF', 'QR', 'GM', 'KP', '3U', 'A9', 'WB', 'V7', 'LA', 'KE', 'ZF', 'IQ', 'GW', 'LG', 'HZ', 'HX', 'ZN', 'YC', '3L', 'VN', 'VJ', 'EI', 'MQ', '8M', 'UA', 'ЮЭ', 'D2', 'MU', 'LO', 'MF', '9B', 'TR', '5N', '2P', 'H9', 'CA', 'VA', '5F', 'MS', 'CI', 'TW', 'KA', 'SA', 'IZ', 'SN', 'YX', 'SV', 'WS', '7G', 'IB', 'U6', '6H', '4G', 'PW', 'RE', 'BG', 'HO', 'GQ', 'AZ', 'EY', 'IO', 'D8', 'JL', 'FV', 'CJ', 'YU', 'V3', 'HV', 'GF', 'GA', 'WZ', 'VF', 'СЛ', 'B2', 'AV', 'PK', 'TG', 'BT', 'ET', 'B6', '7T', 'K6', 'Y7', '6E', 'F6', 'PC', '2L', 'KC', 'NX', 'CD', 'NL', 'LY', 'RO', 'HU', 'KU', 'FZ', 'TK', 'ZH', 'BI', 'BR

In [ ]:
carrier_cols = [col for col in data.columns if "Carrier_code" in col]
unique_carrier_codes = (
    data
    .select(
        pl.concat_list([pl.col(col) for col in carrier_cols])
        .alias("carrier_codes")
    )
    .select(
        pl.col("carrier_codes")
        .list.explode()
        .filter(pl.col("carrier_codes").is_not_null())
        .unique()
    )
    .to_series()
    .to_list()
)
print(unique_carrier_codes)

In [25]:
data['frequentFlyer'].unique()

frequentFlyer
str
"""SU/U6/S7"""
"""- ЮТэйр ЗАО"""
"""SU/UT/S7/DL/LH"""
"""SU/DP/S7/UT"""
"""SU/S7/UT"""
…
"""UT/S7/U6/SU"""
"""SU/AI"""
"""KC/C7/SU"""


Frequent flyer code '- ЮТэйр ЗАО' translates to 'UTair JSC', which has no matching carrier under carrier codes.

In [27]:
# create a list of the unique frequent flyer codes
unique_ff_codes = (
    data.select("frequentFlyer")
    .filter(pl.col("frequentFlyer").is_not_null())  # Remove nulls
    .with_columns(
        pl.col("frequentFlyer").str.split("/").alias("ff_codes")  # Split by "/"
    )
    .select("ff_codes")
    .explode("ff_codes")  # Flatten the lists into individual rows
    .unique()  # Get unique codes
    .sort("ff_codes")  # Optional: sort them
    .to_series()
    .to_list()
)
print(unique_ff_codes)

# let's see if any do not have an associated carrier code
ff_not_in_carrier = set(unique_ff_codes) - set(unique_carrier_codes)
print("FF codes not in carrier list:", ff_not_in_carrier)


['- ЮТэйр ЗАО', '2G', '5N', '6R', '6W', '9X', 'A3', 'A4', 'AA', 'AB', 'AF', 'AI', 'AM', 'AT', 'AY', 'AZ', 'B2', 'BA', 'BD', 'C7', 'CA', 'CX', 'CZ', 'D2', 'D9', 'DL', 'DP', 'DV', 'EK', 'ET', 'EY', 'FV', 'FZ', 'GF', 'HU', 'HY', 'I8', 'J2', 'JP', 'JU', 'KC', 'KE', 'KL', 'LH', 'LO', 'LX', 'MS', 'MU', 'N4', 'OK', 'OS', 'OZ', 'PY', 'QR', 'R3', 'RT', 'S7', 'SK', 'SN', 'SQ', 'SU', 'SV', 'TK', 'U6', 'UA', 'UN', 'UT', 'VS', 'WY', 'Y7']
FF codes not in carrier list: {'AB', 'D9', '2G', 'UN', '- ЮТэйр ЗАО', 'JP', '6W', 'VS', 'BD', 'C7', 'PY', '9X'}


Frequent Flyer codes to Airlines:
* ЮТэйр ЗАО (translates to 'UTair JSC'):  UTair / UT
* AB: Air Berlin / AB
* D9: Daallo Airlines Somalia / D9
* 2G: Northwest Seaplanes / None
* UN: Transaero Airlines / UN (no longer in service)
* JP: Adria Airways / JP (no longer in service)
* 6W: Unknown
* VS: Virgin Atlantic Airways / VS
* BD: BMI British Midland / BD (integrated into British Airways)
* C7: Cinnamon Air / C7
* PY: Surinam Airways / PY
* 9X: Air Columbia / Unknown

So, translate 'ЮТэйр ЗАО' to UT and we'll drop any that don't match an airline IATA code

In [15]:
iata_cols = [col for col in data.columns if "_iata" in col]

unique_iata_codes = (
    data
    .select([
        pl.concat_list([pl.col(col) for col in iata_cols])
        .list.explode()
        .drop_nulls()
        .unique()
        .alias('unique_iata_codes')
    ])
    .get_column('unique_iata_codes')
    .to_list()
)

print(f"Found {len(unique_iata_codes)} unique IATA codes")
print(unique_iata_codes)


Found 679 unique IATA codes
['STW', 'PNH', 'BWN', 'TIF', 'LZY', 'TJM', 'MOW', 'AUH', 'GES', 'CGN', 'OMS', 'SSH', 'EYK', 'YUL', 'DQA', 'CRL', 'XYD', 'LLV', 'KYZ', 'ASB', 'DOY', 'DUR', 'VAN', 'MAD', 'JMK', 'CZL', 'NAL', 'XIY', 'QDU', 'KVX', 'LUN', 'KHV', 'UYN', 'KRK', 'VCP', 'KSQ', 'NOZ', 'ANK', 'ASF', 'KWI', 'IST', 'ORY', 'TMJ', 'NNM', 'NYC', 'EAP', 'SEZ', 'KCY', 'CCU', 'MBA', 'BKO', 'GOX', 'KZN', 'XWC', 'BPE', 'IWA', 'SKG', 'BUS', 'URC', 'LJG', 'BSL', 'VRA', 'NAP', 'HHL', 'BRE', 'BQT', 'ESL', 'VOG', 'ALG', 'QIN', 'DAD', 'IRT', 'KNY', 'BUR', 'GRZ', 'PRG', 'HLD', 'SGN', 'SJW', 'XVQ', 'ADB', 'SHA', 'DYG', 'HOU', 'VRN', 'BLQ', 'OLZ', 'WAW', 'WHA', 'AER', 'PNR', 'HFE', 'KRO', 'NCE', 'JHB', 'HYN', 'PYJ', 'BAH', 'LOS', 'PKV', 'ZVJ', 'SYX', 'SEL', 'WNZ', 'BAK', 'UKX', 'MUC', 'NZG', 'YTO', 'KHH', 'PHL', 'ZDN', 'UCT', 'BZX', 'SHE', 'HMA', 'TSA', 'CIT', 'AGY', 'JGS', 'VNO', 'KUF', 'SCO', 'RAK', 'VCE', 'ZFQ', 'SIN', 'DMS', 'CGR', 'WVB', 'ATL', 'TAS', 'DUB', 'ZYR', 'SZB', 'HSN', 'UGC', 'TYD', 'ADA'

Clean-up
1.

In [6]:
# Variables/Lists used for flight feature engineering and extraction
CUSTOMER_FEATURES = ['profileId', 'companyID', 'sex', 'nationality', 'bySelf', 'isVip']
POLARS_INDEX_COL = ['__index_level_0__']
UNUSED_FLIGHT_FEATURES = [
    'legs0_arrivalAt', 'legs0_departureAt', 'legs1_arrivalAt', 'legs1_departureAt',
    'legs0_duration', 'legs1_duration', 'searchRoute', 'isAccess3D', 'frequentFlyer', 'ff_normalized',
    r'^legs[01]_segments.*$'
]
MAJOR_HUBS = [
    'ATL','DXB','DFW','HND','LHR','DEN','ORD','IST','PVG','ICN','CDG', 'JFK','CLT','MEX','SFO','EWR','MIA','BKK','GRU','HKG'
]


# function to get column groups for the flight segments features that will be used in engineering the new features
def get_column_groups(df: pl.DataFrame) -> Dict[str, List[str]]:
    columns = df.columns
    return {
        'leg0_duration': [col for col in columns if col.startswith('legs0_segments') and col.endswith('_duration')],
        'leg1_duration': [col for col in columns if col.startswith('legs1_segments') and col.endswith('_duration')],
        'mkt_carrier': [col for col in columns if col.endswith('marketingCarrier_code')],
        'op_carrier': [col for col in columns if col.endswith('operatingCarrier_code')],
        'aircraft': [col for col in columns if col.endswith('aircraft_code')],
        'airport': [col for col in columns if 'airport_iata' in col or 'airport_city_iata' in col],
        'seats_avail': [col for col in columns if col.endswith('seatsAvailable')],
        'baggage_type': [col for col in columns if 'baggageAllowance_weightMeasurementType' in col],
        'baggage_allowance': [col for col in columns if 'baggageAllowance_quantity' in col],
        'cabin_class': [col for col in columns if 'cabinClass' in col]
    }


## Creating functions that return Polars expressions, so the feature engineering can be modularized and performed 'lazy' ##
def parse_duration_to_minutes(duration_col: str) -> pl.Expr:
    """Parse duration string to minutes (handles format like '1.02:30:00')."""
    return (
        pl.when(pl.col(duration_col).is_not_null())
        .then(
            pl.col(duration_col).str.extract(r'^(?:(\d+)\.)?', 1).fill_null('0').cast(pl.Int32) * 1440 +
            pl.col(duration_col).str.extract(r'(\d+):(\d+):(\d+)$', 1).fill_null('0').cast(pl.Int32) * 60 +
            pl.col(duration_col).str.extract(r'(\d+):(\d+):(\d+)$', 2).fill_null('0').cast(pl.Int32) +
            (pl.col(duration_col).str.extract(r'(\d+):(\d+):(\d+)$', 3).fill_null('0').cast(pl.Int32) / 60).round(0)
        )
        .otherwise(0)
    )


def create_basic_features(col_groups: Dict[str, List[str]]) -> List[pl.Expr]:
    """Create basic customer and route characteristics features."""
    return [
        # Customer flag features to boolean
        pl.col('isAccess3D').cast(pl.Int32).alias('is_access3D'),

        # Normalized frequentFlyer program, addressing null values as null strings, and translating UT program
        pl.col('frequentFlyer').str.replace('- ЮТэйр ЗАО', 'UT').fill_null('').alias('ff_normalized'),

        # Route characteristics
        pl.col('legs1_departureAt').is_not_null().cast(pl.Int32).alias('is_roundtrip'),
        pl.col('searchRoute').str.slice(0, 3).alias('route_origin'),

        # Hub features
        pl.col('searchRoute').str.slice(0, 3).is_in(MAJOR_HUBS).cast(pl.Int32).alias('origin_is_major_hub'),
        pl.col('searchRoute').str.slice(3, 3).is_in(MAJOR_HUBS).cast(pl.Int32).alias('destination_is_major_hub'),
        pl.max_horizontal([
            pl.col(col).is_in(MAJOR_HUBS) for col in col_groups['airport']
        ]).cast(pl.Int32).alias('includes_major_hub'),
    ]


def create_segment_features(col_groups: Dict[str, List[str]]) -> List[pl.Expr]:
    """Create flight segment related features."""
    return [
        # Number of segments per leg
        pl.concat_list([pl.col(col) for col in col_groups['leg0_duration']])
        .list.drop_nulls().list.len().alias('leg0_num_segments'),

        pl.concat_list([pl.col(col) for col in col_groups['leg1_duration']])
        .list.drop_nulls().list.len().alias('leg1_num_segments'),

        # Total segments
        pl.sum_horizontal([pl.col(col).is_not_null() for col in col_groups['aircraft']])
        .alias('total_segments'),

        # Flight time in minutes (sum of segment durations)
        pl.sum_horizontal([
            pl.col(col).str.extract(r'^(\d+):(\d+)', 1).cast(pl.Int32, strict=False).fill_null(0).mul(60)
            .add(pl.col(col).str.extract(r'^(\d+):(\d+)', 2).cast(pl.Int32, strict=False).fill_null(0))
            for col in col_groups['leg0_duration']
        ]).alias('leg0_flight_time_min'),

        pl.sum_horizontal([
            pl.col(col).str.extract(r'^(\d+):(\d+)', 1).cast(pl.Int32, strict=False).fill_null(0).mul(60)
            .add(pl.col(col).str.extract(r'^(\d+):(\d+)', 2).cast(pl.Int32, strict=False).fill_null(0))
            for col in col_groups['leg1_duration']
        ]).alias('leg1_flight_time_min'),
    ]


def create_time_features() -> List[pl.Expr]:
    """Create time-based features."""
    return [
        # Booking lead time
        # ((pl.col('legs0_departureAt').str.to_datetime().cast(pl.Datetime) -
        #   pl.col('requestDate').cast(pl.Datetime)) / pl.duration(minutes=1))
        # .cast(pl.Int32).alias('booking_lead_time_min'),

        # Booking lead time in whole days
        ((pl.col('legs0_departureAt').str.to_datetime().cast(pl.Datetime) -
          pl.col('requestDate').cast(pl.Datetime)) / pl.duration(days=1))
        .cast(pl.Int32).alias('booking_lead_time_days'),

        # Duration features
        parse_duration_to_minutes('legs0_duration').alias('leg0_duration_min'),
        parse_duration_to_minutes('legs1_duration').alias('leg1_duration_min'),

        # Departure/arrival hours
        pl.col('legs0_departureAt').str.to_datetime().dt.hour().alias('leg0_departure_hour'),
        pl.col('legs0_arrivalAt').str.to_datetime().dt.hour().alias('leg0_arrival_hour'),
        pl.col('legs1_departureAt').str.to_datetime().dt.hour().alias('leg1_departure_hour'),
        pl.col('legs1_arrivalAt').str.to_datetime().dt.hour().alias('leg1_arrival_hour'),
    ]


def create_carrier_features(col_groups: Dict[str, List[str]]) -> List[pl.Expr]:
    """Create carrier and aircraft features."""
    # Create combined carrier column group for carrier features
    all_carrier_cols = col_groups['mkt_carrier'] + col_groups['op_carrier']

    # Create frequent flyer matching expressions
    ff_matches_mkt = [
        pl.when(pl.col(col).is_not_null() & (pl.col(col) != ''))
        .then(pl.col('ff_normalized').str.contains(pl.col(col)))
        .otherwise(False)
        for col in col_groups['mkt_carrier']
    ]
    ff_matches_op = [
        pl.when(pl.col(col).is_not_null() & (pl.col(col) != ''))
        .then(pl.col('ff_normalized').str.contains(pl.col(col)))
        .otherwise(False)
        for col in col_groups['op_carrier']
    ]

    return [
        # Carrier features
        pl.concat_list([pl.col(col) for col in all_carrier_cols])
        .list.drop_nulls().list.unique().list.len().alias('unique_carriers'),

        pl.coalesce([pl.col(col) for col in all_carrier_cols]).alias('primary_carrier'),
        pl.col('legs0_segments0_marketingCarrier_code').alias('marketing_carrier'),

        # Frequent flyer matching (check if FF programs match carriers)
        pl.max_horizontal(ff_matches_mkt).cast(pl.Int32).alias('has_mkt_ff'),
        pl.max_horizontal(ff_matches_op).cast(pl.Int32).alias('has_op_ff'),

        # Aircraft features
        pl.concat_list([pl.col(col) for col in col_groups['aircraft']])
        .list.drop_nulls().list.unique().list.len().alias('aircraft_diversity'),

        pl.coalesce([pl.col(col) for col in col_groups['aircraft']]).alias('primary_aircraft'),
    ]


def create_service_features(col_groups: Dict[str, List[str]]) -> List[pl.Expr]:
    """Create service-related features (fees, cabin, seats, baggage)."""
    return [
        # Cancellation/exchange fees
        (
            ((pl.col('miniRules0_monetaryAmount') > 0) & pl.col('miniRules0_monetaryAmount').is_not_null()) |
            ((pl.col('miniRules0_percentage') > 0) & pl.col('miniRules0_percentage').is_not_null())
        ).cast(pl.Int32).alias('has_cancellation_fee'),

        (
            ((pl.col('miniRules1_monetaryAmount') > 0) & pl.col('miniRules1_monetaryAmount').is_not_null()) |
            ((pl.col('miniRules1_percentage') > 0) & pl.col('miniRules1_percentage').is_not_null())
        ).cast(pl.Int32).alias('has_exchange_fee'),

        # Cabin class features
        pl.max_horizontal([pl.col(col) for col in col_groups['cabin_class']]).alias('max_cabin_class'),
        pl.mean_horizontal([pl.col(col) for col in col_groups['cabin_class']]).alias('avg_cabin_class'),

        # Seat availability (using for understanding popularity/scarcity)
        pl.min_horizontal([pl.col(col) for col in col_groups['seats_avail']]).alias('min_seats_available'),
        pl.sum_horizontal([pl.col(col).fill_null(0) for col in col_groups['seats_avail']]).alias('total_seats_available'),

        # Baggage allowance
        pl.min_horizontal([
            pl.when(pl.col(type_col) == 0).then(pl.col(qty_col)).otherwise(pl.lit(None))
            for type_col, qty_col in zip(col_groups['baggage_type'], col_groups['baggage_allowance'])
        ]).fill_null(0).alias("baggage_allowance_quantity"),

        pl.min_horizontal([
            pl.when(pl.col(type_col) == 1).then(pl.col(qty_col)).otherwise(pl.lit(None))
            for type_col, qty_col in zip(col_groups['baggage_type'], col_groups['baggage_allowance'])
        ]).fill_null(0).alias("baggage_allowance_weight")
    ]


def add_route_popularity(lazy_df: pl.LazyFrame) -> pl.LazyFrame:
    """Add search route popularity features."""
    return lazy_df.with_columns([
        pl.len().over('searchRoute').alias('route_popularity')
    ]).with_columns([
        pl.col('route_popularity').log1p().alias('route_popularity_log')
    ])


def extract_flight_features(df: pl.DataFrame) -> pl.DataFrame:
    """ Aggregate function for processing feature extraction functions"""
    # Determine if dataframe already processed by checking for a calculated feature
    if 'total_duration' in df.columns:
        return df

    # Get column groups
    col_groups = get_column_groups(df)

    # Start with lazy frame, dropping unnecessary columns
    lazy_df = df.drop(CUSTOMER_FEATURES + POLARS_INDEX_COL, strict=False).lazy()

    # Apply all feature groups
    lazy_df = lazy_df.with_columns([
        *create_basic_features(col_groups),
        *create_segment_features(col_groups),
        *create_time_features(),
        *create_service_features(col_groups)
    ])

    # Add carrier features (requires ff_normalized from create_basic_features() step)
    lazy_df = lazy_df.with_columns(create_carrier_features(col_groups))

    # Add total duration (depends on leg durations calculated in create_time_features() above)
    lazy_df = lazy_df.with_columns([
        pl.when(pl.col('leg1_duration_min').is_null())
        .then(pl.col('leg0_duration_min'))
        .otherwise(pl.col('leg0_duration_min') + pl.col('leg1_duration_min'))
        .alias('total_duration')
    ])

    # Add route popularity
    lazy_df = add_route_popularity(lazy_df)

    # Materialize to generate new features, drop unused features, and fill nulls
    return (lazy_df
            .select(pl.exclude(UNUSED_FLIGHT_FEATURES))
            .fill_null(0)
            .collect())


In [7]:
data = extract_flight_features(data)

In [8]:
data.head(300)

Id,corporateTariffCode,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,ranker_id,requestDate,taxes,totalPrice,is_vip,is_access3D,is_roundtrip,route_origin,origin_is_major_hub,destination_is_major_hub,includes_major_hub,leg0_num_segments,leg1_num_segments,total_segments,leg0_flight_time_min,leg1_flight_time_min,booking_lead_time_min,leg0_duration_min,leg1_duration_min,leg0_departure_hour,leg0_arrival_hour,leg1_departure_hour,leg1_arrival_hour,has_cancellation_fee,has_exchange_fee,max_cabin_class,avg_cabin_class,min_seats_available,total_seats_available,baggage_allowance_quantity,baggage_allowance_weight,unique_carriers,primary_carrier,marketing_carrier,has_mkt_ff,has_op_ff,aircraft_diversity,primary_aircraft,total_duration,route_popularity,route_popularity_log
i64,i64,f64,f64,f64,f64,f64,f64,f64,i64,str,datetime[ns],f64,f64,i32,i32,i32,str,i32,i32,i32,u32,u32,u32,i32,i32,i32,f64,f64,i8,i8,i8,i8,i32,i32,f64,f64,f64,f64,f64,f64,u32,str,str,i32,i32,u32,str,f64,u32,f64
0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,370.0,16884.0,0,0,1,"""TLK""",0,0,0,1,1,2,160,155,42516,160.0,155.0,15,16,9,14,0,0,1.0,1.0,9.0,18.0,1.0,0.0,1,"""KV""","""KV""",0,0,1,"""YK2""",315.0,25,3.258097
1,123,2300.0,0.0,1.0,3500.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,2240.0,51125.0,0,1,1,"""TLK""",0,0,0,2,2,4,250,245,42141,445.0,505.0,9,14,22,8,1,1,1.0,1.0,4.0,26.0,1.0,0.0,1,"""S7""","""S7""",1,1,1,"""E70""",950.0,25,3.258097
2,0,2300.0,0.0,1.0,3500.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,2240.0,53695.0,0,0,1,"""TLK""",0,0,0,2,2,4,250,245,42141,445.0,505.0,9,14,22,8,1,1,1.0,1.0,4.0,26.0,1.0,0.0,1,"""S7""","""S7""",1,1,1,"""E70""",950.0,25,3.258097
3,123,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,2240.0,81880.0,0,1,1,"""TLK""",0,0,0,2,2,4,250,245,42141,445.0,505.0,9,14,22,8,0,0,1.0,1.0,4.0,26.0,1.0,0.0,1,"""S7""","""S7""",1,1,1,"""E70""",950.0,25,3.258097
4,0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,2240.0,86070.0,0,0,1,"""TLK""",0,0,0,2,2,4,250,245,42141,445.0,505.0,9,14,22,8,0,0,1.0,1.0,4.0,26.0,1.0,0.0,1,"""S7""","""S7""",1,1,1,"""E70""",950.0,25,3.258097
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
295,0,2300.0,0.0,1.0,3500.0,0.0,1.0,1.0,1,"""0e51002eda914a39a9309cecf0a659…",2024-05-17 04:47:52,720.0,7420.0,0,0,0,"""OVB""",0,0,0,1,0,1,80,0,22372,80.0,0.0,17,19,0,0,1,1,1.0,1.0,9.0,9.0,1.0,0.0,1,"""S7""","""S7""",0,0,1,"""E70""",80.0,4522,8.416931
296,115,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1,"""0e51002eda914a39a9309cecf0a659…",2024-05-17 04:47:52,720.0,17040.0,0,1,0,"""OVB""",0,0,0,1,0,1,80,0,22372,80.0,0.0,17,19,0,0,0,0,1.0,1.0,9.0,9.0,1.0,0.0,1,"""S7""","""S7""",0,0,1,"""E70""",80.0,4522,8.416931
297,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1,"""0e51002eda914a39a9309cecf0a659…",2024-05-17 04:47:52,720.0,18080.0,0,0,0,"""OVB""",0,0,0,1,0,1,80,0,22372,80.0,0.0,17,19,0,0,0,0,1.0,1.0,9.0,9.0,1.0,0.0,1,"""S7""","""S7""",0,0,1,"""E70""",80.0,4522,8.416931


In [20]:
data.filter(pl.col('is_vip') == 1).limit(100)

id,corporate_tariff_code,mini_rules0_monetary_amount,mini_rules0_percentage,mini_rules0_status_infos,mini_rules1_monetary_amount,mini_rules1_percentage,mini_rules1_status_infos,pricing_info_is_access_tp,pricing_info_passenger_count,ranker_id,request_date,taxes,total_price,is_vip,is_access3_d,is_roundtrip,route_origin,origin_is_major_hub,destination_is_major_hub,includes_major_hub,leg0_num_segments,leg1_num_segments,total_segments,leg0_flight_time_min,leg1_flight_time_min,booking_lead_time_min,leg0_duration_min,leg1_duration_min,leg0_departure_hour,leg0_arrival_hour,leg1_departure_hour,leg1_arrival_hour,has_cancellation_fee,has_exchange_fee,max_cabin_class,avg_cabin_class,min_seats_available,total_seats_available,baggage_allowance_quantity,baggage_allowance_weight,unique_carriers,primary_carrier,marketing_carrier,has_mkt_ff,has_op_ff,aircraft_diversity,primary_aircraft,total_duration,route_popularity,route_popularity_log,price_rank,price_percentile,price_zscore,options_count,price_ratio_to_min,price_ratio_to_max,price_ratio_to_median,is_cheapest,is_most_expensive,is_below_median,price_range,price_std,price_cv,price_gap_to_next_cheapest
i64,i64,f64,f64,f64,f64,f64,f64,f64,i64,str,datetime[ns],f64,f64,i32,i32,i32,str,i32,i32,i32,u32,u32,u32,i32,i32,i32,f64,f64,i8,i8,i8,i8,i32,i32,f64,f64,f64,f64,f64,f64,u32,str,str,i32,i32,u32,str,f64,u32,f64,u32,f64,f64,u32,f64,f64,f64,i32,i32,i32,f64,f64,f64,f64
6046,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,"""b728143078f0455f856d14e4388d93…",2024-05-17 09:12:57,643.0,11143.0,1,0,0,"""MOW""",0,0,0,1,0,1,225,0,46417,225.0,0.0,14,18,0,0,0,0,1.0,1.0,9.0,9.0,0.0,0.0,1,"""N4""","""N4""",0,0,1,"""333""",225.0,140074,11.849933,1,0.0,-1.079646,197,1.0,0.134975,0.579369,1,0,1,71413.0,11174.110687,0.481496,58.0
6047,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,"""b728143078f0455f856d14e4388d93…",2024-05-17 09:12:57,643.0,13243.0,1,0,0,"""MOW""",0,0,0,1,0,1,225,0,46417,225.0,0.0,14,18,0,0,0,0,1.0,1.0,9.0,9.0,1.0,0.0,1,"""N4""","""N4""",0,0,1,"""333""",225.0,140074,11.849933,12,5.612245,-0.891711,197,1.188459,0.160412,0.688556,0,0,1,71413.0,11174.110687,0.481496,-973.0
6048,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,"""b728143078f0455f856d14e4388d93…",2024-05-17 09:12:57,643.0,16143.0,1,0,0,"""MOW""",0,0,0,1,0,1,225,0,46417,225.0,0.0,14,18,0,0,0,0,1.0,1.0,9.0,9.0,1.0,0.0,1,"""N4""","""N4""",0,0,1,"""333""",225.0,140074,11.849933,43,21.428571,-0.632183,197,1.448712,0.19554,0.839339,0,0,1,71413.0,11174.110687,0.481496,-3869.0
6049,0,4600.0,0.0,1.0,0.0,0.0,0.0,1.0,1,"""b728143078f0455f856d14e4388d93…",2024-05-17 09:12:57,563.0,18623.0,1,0,0,"""MOW""",0,0,0,1,0,1,200,0,45657,200.0,0.0,2,5,0,0,1,0,1.0,1.0,1.0,1.0,0.0,0.0,1,"""SU""","""SU""",0,0,1,"""32B""",200.0,140074,11.849933,76,38.265306,-0.410241,197,1.671273,0.22558,0.968284,0,0,1,71413.0,11174.110687,0.481496,-6349.0
6050,0,4600.0,0.0,1.0,4600.0,0.0,1.0,1.0,1,"""b728143078f0455f856d14e4388d93…",2024-05-17 09:12:57,563.0,22223.0,1,0,0,"""MOW""",0,0,0,1,0,1,200,0,45657,200.0,0.0,2,5,0,0,1,1,1.0,1.0,1.0,1.0,1.0,0.0,1,"""SU""","""SU""",0,0,1,"""32B""",200.0,140074,11.849933,122,61.734694,-0.088068,197,1.994346,0.269187,1.155462,0,0,0,71413.0,11174.110687,0.481496,-9949.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
6141,0,4600.0,0.0,1.0,4600.0,0.0,1.0,1.0,1,"""b728143078f0455f856d14e4388d93…",2024-05-17 09:12:57,563.0,19233.0,1,0,0,"""MOW""",0,0,0,1,0,1,230,0,46702,230.0,0.0,19,23,0,0,1,1,1.0,1.0,1.0,1.0,1.0,0.0,1,"""SU""","""SU""",0,0,1,"""32A""",230.0,140074,11.849933,90,45.408163,-0.355651,197,1.726016,0.232969,1.0,0,0,0,71413.0,11174.110687,0.481496,0.0
6142,0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,"""b728143078f0455f856d14e4388d93…",2024-05-17 09:12:57,563.0,25433.0,1,0,0,"""MOW""",0,0,0,1,0,1,230,0,46702,230.0,0.0,19,23,0,0,0,0,1.0,1.0,1.0,1.0,2.0,0.0,1,"""SU""","""SU""",0,0,1,"""32A""",230.0,140074,11.849933,145,73.469388,0.199203,197,2.282419,0.30807,1.322363,0,0,0,71413.0,11174.110687,0.481496,-6200.0
6143,

In [9]:
def add_search_price_features(df):

    lazy_df = df.lazy().with_columns([
        # Basic ranking and percentile features
        pl.col('totalPrice').rank(method='min').over('ranker_id').alias('price_rank'),

        # Price percentile (0-100, lower = cheaper)
        ((pl.col('totalPrice').rank(method='min').over('ranker_id') - 1) /
         (pl.col('totalPrice').count().over('ranker_id') - 1) * 100)
        .fill_null(50.0).alias('price_percentile'),

        # Z-score within search group
        pl.when(pl.col('totalPrice').std().over('ranker_id') > 0)
        .then(
            (pl.col('totalPrice') - pl.col('totalPrice').mean().over('ranker_id')) /
            pl.col('totalPrice').std().over('ranker_id')
        )
        .otherwise(0.0)
        .alias('price_zscore'),

        # Group statistics for ratio calculations
        pl.col('totalPrice').min().over('ranker_id').alias('_min_price'),
        pl.col('totalPrice').max().over('ranker_id').alias('_max_price'),
        pl.col('totalPrice').median().over('ranker_id').alias('_median_price'),
        pl.col('totalPrice').mean().over('ranker_id').alias('_mean_price'),
        pl.col('totalPrice').std().over('ranker_id').alias('_std_price'),
        pl.col('totalPrice').count().over('ranker_id').alias('options_count'),

    ]).with_columns([
        # Ratio features
        (pl.col('totalPrice') / pl.col('_min_price')).alias('price_ratio_to_min'),
        (pl.col('totalPrice') / pl.col('_max_price')).alias('price_ratio_to_max'),
        (pl.col('totalPrice') / pl.col('_median_price')).alias('price_ratio_to_median'),

        # Binary position features
        (pl.col('totalPrice') == pl.col('_min_price')).cast(pl.Int32).alias('is_cheapest'),
        (pl.col('totalPrice') == pl.col('_max_price')).cast(pl.Int32).alias('is_most_expensive'),
        (pl.col('totalPrice') < pl.col('_median_price')).cast(pl.Int32).alias('is_below_median'),

        # Price range and variability
        (pl.col('_max_price') - pl.col('_min_price')).alias('price_range'),
        pl.col('_std_price').alias('price_std'),

        # Coefficient of variation
        pl.when(pl.col('_mean_price') > 0)
        .then(pl.col('_std_price') / pl.col('_mean_price'))
        .otherwise(0.0)
        .alias('price_cv'),

    ]).with_columns([
        # Sort prices to calculate price gap
        pl.col('totalPrice').sort().over('ranker_id').alias('_sorted_prices'),

    ]).with_columns([
        # Calculate price gap using window functions
        pl.when(pl.col('price_rank') < pl.col('options_count'))
        .then(
            # Get the next higher price in the sorted list
            pl.col('_sorted_prices').shift(-1).over('ranker_id') - pl.col('totalPrice')
        )
        .otherwise(0.0)
        .alias('price_gap_to_next_cheapest')

    ]).drop(['^_.*$']).fill_null(0)  # Remove the temporary columns

    return lazy_df.collect()

In [10]:
data = add_search_price_features(data)

In [11]:
print(f"shape: {data.shape}")
print(f"data types: {data.dtypes}")
data.head(20)

shape: (18145372, 65)
data types: [Int64, Int64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Int64, String, Datetime(time_unit='ns', time_zone=None), Float64, Float64, Int32, Int32, Int32, String, Int32, Int32, Int32, UInt32, UInt32, UInt32, Int32, Int32, Int32, Float64, Float64, Int8, Int8, Int8, Int8, Int32, Int32, Float64, Float64, Float64, Float64, Float64, Float64, UInt32, String, String, Int32, Int32, UInt32, String, Float64, UInt32, Float64, UInt32, Float64, Float64, UInt32, Float64, Float64, Float64, Int32, Int32, Int32, Float64, Float64, Float64, Float64]


Id,corporateTariffCode,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,ranker_id,requestDate,taxes,totalPrice,is_vip,is_access3D,is_roundtrip,route_origin,origin_is_major_hub,destination_is_major_hub,includes_major_hub,leg0_num_segments,leg1_num_segments,total_segments,leg0_flight_time_min,leg1_flight_time_min,booking_lead_time_min,leg0_duration_min,leg1_duration_min,leg0_departure_hour,leg0_arrival_hour,leg1_departure_hour,leg1_arrival_hour,has_cancellation_fee,has_exchange_fee,max_cabin_class,avg_cabin_class,min_seats_available,total_seats_available,baggage_allowance_quantity,baggage_allowance_weight,unique_carriers,primary_carrier,marketing_carrier,has_mkt_ff,has_op_ff,aircraft_diversity,primary_aircraft,total_duration,route_popularity,route_popularity_log,price_rank,price_percentile,price_zscore,options_count,price_ratio_to_min,price_ratio_to_max,price_ratio_to_median,is_cheapest,is_most_expensive,is_below_median,price_range,price_std,price_cv,price_gap_to_next_cheapest
i64,i64,f64,f64,f64,f64,f64,f64,f64,i64,str,datetime[ns],f64,f64,i32,i32,i32,str,i32,i32,i32,u32,u32,u32,i32,i32,i32,f64,f64,i8,i8,i8,i8,i32,i32,f64,f64,f64,f64,f64,f64,u32,str,str,i32,i32,u32,str,f64,u32,f64,u32,f64,f64,u32,f64,f64,f64,i32,i32,i32,f64,f64,f64,f64
0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,370.0,16884.0,0,0,1,"""TLK""",0,0,0,1,1,2,160,155,42516,160.0,155.0,15,16,9,14,0,0,1.0,1.0,9.0,18.0,1.0,0.0,1,"""KV""","""KV""",0,0,1,"""YK2""",315.0,25,3.258097,1,0.0,-2.605417,25,1.0,0.196166,0.314443,1,0,1,69186.0,18905.2881,0.285837,34241.0
1,123,2300.0,0.0,1.0,3500.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,2240.0,51125.0,0,1,1,"""TLK""",0,0,0,2,2,4,250,245,42141,445.0,505.0,9,14,22,8,1,1,1.0,1.0,4.0,26.0,1.0,0.0,1,"""S7""","""S7""",1,1,1,"""E70""",950.0,25,3.258097,2,4.166667,-0.794231,25,3.028015,0.593993,0.952137,0,0,1,69186.0,18905.2881,0.285837,0.0
2,0,2300.0,0.0,1.0,3500.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,2240.0,53695.0,0,0,1,"""TLK""",0,0,0,2,2,4,250,245,42141,445.0,505.0,9,14,22,8,1,1,1.0,1.0,4.0,26.0,1.0,0.0,1,"""S7""","""S7""",1,1,1,"""E70""",950.0,25,3.258097,8,29.166667,-0.65829,25,3.18023,0.623853,1.0,0,0,0,69186.0,18905.2881,0.285837,-2570.0
3,123,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,2240.0,81880.0,0,1,1,"""TLK""",0,0,0,2,2,4,250,245,42141,445.0,505.0,9,14,22,8,0,0,1.0,1.0,4.0,26.0,1.0,0.0,1,"""S7""","""S7""",1,1,1,"""E70""",950.0,25,3.258097,14,54.166667,0.832563,25,4.849562,0.951319,1.524909,0,0,0,69186.0,18905.2881,0.285837,-30755.0
4,0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,2240.0,86070.0,0,0,1,"""TLK""",0,0,0,2,2,4,250,245,42141,445.0,505.0,9,14,22,8,0,0,1.0,1.0,4.0,26.0,1.0,0.0,1,"""S7""","""S7""",1,1,1,"""E70""",950.0,25,3.258097,20,79.166667,1.054194,25,5.097726,1.0,1.602943,0,1,0,69186.0,18905.2881,0.285837,-34945.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
15,123,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,2240.0,81880.0,0,1,1,"""TLK""",0,0,0,2,2,4,250,245,42141,445.0,755.0,9,14,17,8,0,0,1.0,1.0,4.0,26.0,1.0,0.0,1,"""S7""","""S7""",1,1,1,"""E70""",1200.0,25,3.258097,14,54.166667,0.832563,25,4.849562,0.951319,1.524909,0,0,0,69186.0,18905.2881,0.285837,0.0
16,0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,2240.0,86070.0,0,0,1,"""TLK""",0,0,0,2,2,4,250,245,42141,445.0,755.0,9,14,17,8,0,0,1.0,1.0,4.0,26.0,1.0,0.0,1,"""S7""","""S7""",1,1,1,"""E70""",1200.0,25,3.258097,20,79.166667,1.054194,25,5.097726,1.0,1.602943,0,1,0,69186.0,18905.2881,0.285837,-4190.0
17,123,2300.0,0.0,1.0,3500.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b

In [4]:
def camel_to_snake(name):
    """Convert camelCase or PascalCase to snake_case"""
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    s2 = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1)
    return s2.lower()

def convert_columns_to_snake_case(df):
    """Convert all column names in a polars DataFrame to snake_case"""
    return df.rename({col: camel_to_snake(col) for col in df.columns})

In [13]:
data = convert_columns_to_snake_case(data)
data.head(100)

id,corporate_tariff_code,mini_rules0_monetary_amount,mini_rules0_percentage,mini_rules0_status_infos,mini_rules1_monetary_amount,mini_rules1_percentage,mini_rules1_status_infos,pricing_info_is_access_tp,pricing_info_passenger_count,ranker_id,request_date,taxes,total_price,is_vip,is_access3_d,is_roundtrip,route_origin,origin_is_major_hub,destination_is_major_hub,includes_major_hub,leg0_num_segments,leg1_num_segments,total_segments,leg0_flight_time_min,leg1_flight_time_min,booking_lead_time_min,leg0_duration_min,leg1_duration_min,leg0_departure_hour,leg0_arrival_hour,leg1_departure_hour,leg1_arrival_hour,has_cancellation_fee,has_exchange_fee,max_cabin_class,avg_cabin_class,min_seats_available,total_seats_available,baggage_allowance_quantity,baggage_allowance_weight,unique_carriers,primary_carrier,marketing_carrier,has_mkt_ff,has_op_ff,aircraft_diversity,primary_aircraft,total_duration,route_popularity,route_popularity_log,price_rank,price_percentile,price_zscore,options_count,price_ratio_to_min,price_ratio_to_max,price_ratio_to_median,is_cheapest,is_most_expensive,is_below_median,price_range,price_std,price_cv,price_gap_to_next_cheapest
i64,i64,f64,f64,f64,f64,f64,f64,f64,i64,str,datetime[ns],f64,f64,i32,i32,i32,str,i32,i32,i32,u32,u32,u32,i32,i32,i32,f64,f64,i8,i8,i8,i8,i32,i32,f64,f64,f64,f64,f64,f64,u32,str,str,i32,i32,u32,str,f64,u32,f64,u32,f64,f64,u32,f64,f64,f64,i32,i32,i32,f64,f64,f64,f64
0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,370.0,16884.0,0,0,1,"""TLK""",0,0,0,1,1,2,160,155,42516,160.0,155.0,15,16,9,14,0,0,1.0,1.0,9.0,18.0,1.0,0.0,1,"""KV""","""KV""",0,0,1,"""YK2""",315.0,25,3.258097,1,0.0,-2.605417,25,1.0,0.196166,0.314443,1,0,1,69186.0,18905.2881,0.285837,34241.0
1,123,2300.0,0.0,1.0,3500.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,2240.0,51125.0,0,1,1,"""TLK""",0,0,0,2,2,4,250,245,42141,445.0,505.0,9,14,22,8,1,1,1.0,1.0,4.0,26.0,1.0,0.0,1,"""S7""","""S7""",1,1,1,"""E70""",950.0,25,3.258097,2,4.166667,-0.794231,25,3.028015,0.593993,0.952137,0,0,1,69186.0,18905.2881,0.285837,0.0
2,0,2300.0,0.0,1.0,3500.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,2240.0,53695.0,0,0,1,"""TLK""",0,0,0,2,2,4,250,245,42141,445.0,505.0,9,14,22,8,1,1,1.0,1.0,4.0,26.0,1.0,0.0,1,"""S7""","""S7""",1,1,1,"""E70""",950.0,25,3.258097,8,29.166667,-0.65829,25,3.18023,0.623853,1.0,0,0,0,69186.0,18905.2881,0.285837,-2570.0
3,123,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,2240.0,81880.0,0,1,1,"""TLK""",0,0,0,2,2,4,250,245,42141,445.0,505.0,9,14,22,8,0,0,1.0,1.0,4.0,26.0,1.0,0.0,1,"""S7""","""S7""",1,1,1,"""E70""",950.0,25,3.258097,14,54.166667,0.832563,25,4.849562,0.951319,1.524909,0,0,0,69186.0,18905.2881,0.285837,-30755.0
4,0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,2240.0,86070.0,0,0,1,"""TLK""",0,0,0,2,2,4,250,245,42141,445.0,505.0,9,14,22,8,0,0,1.0,1.0,4.0,26.0,1.0,0.0,1,"""S7""","""S7""",1,1,1,"""E70""",950.0,25,3.258097,20,79.166667,1.054194,25,5.097726,1.0,1.602943,0,1,0,69186.0,18905.2881,0.285837,-34945.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
95,0,2300.0,0.0,1.0,3500.0,0.0,1.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",2024-05-17 03:40:53,1235.0,89570.0,0,0,0,"""KHV""",0,0,0,2,0,2,645,0,4584,1410.0,0.0,8,8,0,0,1,1,2.0,1.5,3.0,12.0,1.0,0.0,1,"""S7""","""S7""",0,0,1,"""321""",1410.0,291,5.676754,26,75.757576,0.75143,34,12.361303,0.536765,2.543482,0,0,0,159624.0,46232.122167,0.843193,3305.0
96,139,4000.0,0.0,1.0,0.0,0.0,0.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",2024-05-17 03:40:53,1235.0,92875.0,0,1,0,"""KHV""",0,0,0,2,0,2,645,0,4584,1410.0,0.0,8,8,0,0,1,0,2.0,2.0,3.0,6.0,1.0,0.0,1,"""S7""","""S7""",0,0,1,"""321""",1410.0,291,5.676754,27,78.787879,0.822917,34,12.817417,0.556571,2.637333,0,0,0,159624.0,46232.122167,0.843193,3815.0
97,0,4000.0,0.0,1.0,0.0,0.0,0.0

In [3]:
def prepare_features_for_clustering(df, drop_cols=None):
    # Features to drop
    if drop_cols is None:
        drop_cols = ['id', 'ranker_id', 'request_date']

    # Identify categorical features (non-numeric columns)
    categorical_features = [col for col, dtype in df.schema.items() if not dtype.is_numeric()]

    # Using lazy computations to avoid memory issues
    lazy_df = df.lazy()

    # Encode categorical features
    encoding_expressions = []
    encoders = {}  # Mapping of feature name to encoding dictionary so we can evaluate cluster assignments

    for feature in categorical_features:
        # Get unique values for the feature
        unique_values = df[feature].fill_null('MISSING').unique().sort().to_list()

        # Create mapping dictionary
        encoders[feature] = {val: idx for idx, val in enumerate(unique_values)}

        # Convert to categorical then to physical (integer codes)
        encoding_expr = (
            pl.col(feature)
            .fill_null('MISSING')
            .cast(pl.Categorical)
            .to_physical()
            .alias(f'{feature}_encoded')
        )
        encoding_expressions.append(encoding_expr)

    # Apply encodings and exclude original categorical features
    result_df = lazy_df.with_columns(encoding_expressions).select(pl.exclude(categorical_features + drop_cols)).fill_null(0).collect()

    return result_df, encoders

In [15]:
# Encode categorical features
clustering_df, encoders = prepare_features_for_clustering(data)

# Convert to Pandas DataFrame for StandardScaler
clustering_pd_df = clustering_df.to_pandas().fillna(0)

In [16]:
# Scale the features for use with KMeans
scaler = StandardScaler()
features_scaled = scaler.fit_transform(clustering_pd_df)

del clustering_pd_df
gc.collect()

0

In [17]:
# Use MiniBatchKMeans to cluster the data (more efficient for very large datasets)
kmeans = MiniBatchKMeans(n_clusters=30, batch_size=50000, random_state=42)
clusters = kmeans.fit_predict(features_scaled)

In [18]:
def interpret_clusters(original_df, clusters, n_clusters_to_show=10):
    # Add clusters to Polars dataframe
    df_with_clusters = original_df.with_columns([
        pl.Series("cluster", clusters)
    ])

    print(f"=== Cluster Analysis ===")
    print(f"Total clusters: {len(np.unique(clusters))}")
    print(f"Dataset size: {len(df_with_clusters):,} rows")

    for cluster_id in range(min(n_clusters_to_show, len(np.unique(clusters)))):
        # Filter to cluster using Polars
        cluster_data = df_with_clusters.filter(pl.col("cluster") == cluster_id)
        cluster_size = len(cluster_data)

        print(f"\n{'='*50}")
        print(f"CLUSTER {cluster_id} ({cluster_size:,} flights, {cluster_size/len(df_with_clusters)*100:.1f}%)")
        print(f"{'='*50}")

        # Show top categorical features using Polars operations
        if 'primary_carrier' in cluster_data.columns:
            print(f"\nTop Carriers:")
            top_carriers = cluster_data.group_by('primary_carrier').agg(pl.len().alias('count')).sort('count', descending=True).head()
            for row in top_carriers.iter_rows(named=True):
                carrier, count = row['primary_carrier'], row['count']
                pct = count/cluster_size*100
                print(f"  {carrier}: {count:,} ({pct:.1f}%)")

        # Show route popularity
        if 'route_popularity' in cluster_data.columns:
            avg_route_popularity = cluster_data.select(pl.col('route_popularity').mean()).item()
            print(f"\nRoute Popularity: {avg_route_popularity:.1f} (avg searches per route)")

        # Numerical statistics using Polars
        print(f"\nFlight Characteristics:")
        stats = cluster_data.select([
            pl.col('total_price').mean().alias('avg_price'),
            pl.col('total_price').median().alias('med_price'),
            pl.col('total_duration').mean().alias('avg_duration'),
            pl.col('total_segments').mean().alias('avg_segments'),
            pl.col('is_roundtrip').mean().alias('roundtrip_pct'),
            pl.col('booking_lead_time_min').mean().alias('avg_lead_time_min'),
            pl.col('price_percentile').mean().alias('avg_price_percentile')
        ]).to_pandas().iloc[0]

        print(f"  Avg Price: ${stats['avg_price']:,.0f} (${stats['med_price']:,.0f} median)")
        print(f"  Avg Duration: {stats['avg_duration']:.0f} min ({stats['avg_duration']/60:.1f} hrs)")
        print(f"  Avg Segments: {stats['avg_segments']:.1f}")
        print(f"  Roundtrip: {stats['roundtrip_pct']*100:.1f}%")
        print(f"  Avg Booking Lead Time: {stats['avg_lead_time_min']/1440:.1f} days")
        print(f"  Avg Price Percentile: {stats['avg_price_percentile']:.1f}% (lower is cheaper)")

        # Frequent Flyer Program Match
        if 'has_mkt_ff' in cluster_data.columns and 'has_op_ff' in cluster_data.columns:
            ff_stats = cluster_data.select([
                pl.col('has_mkt_ff').mean().alias('mkt_ff_match_pct'),
                pl.col('has_op_ff').mean().alias('op_ff_match_pct'),
                (pl.col('has_mkt_ff') | pl.col('has_op_ff')).mean().alias('any_ff_match_pct')
            ]).to_pandas().iloc[0]

            print(f"\nFrequent Flyer Program Match:")
            print(f"  Marketing Carrier FF Match: {ff_stats['mkt_ff_match_pct']*100:.1f}%")
            print(f"  Operating Carrier FF Match: {ff_stats['op_ff_match_pct']*100:.1f}%")
            print(f"  Any Carrier FF Match: {ff_stats['any_ff_match_pct']*100:.1f}%")

        # Service level statistics
        service_stats = cluster_data.select([
            pl.col('has_cancellation_fee').mean().alias('cancellation_fee_pct'),
            pl.col('has_exchange_fee').mean().alias('exchange_fee_pct'),
            pl.col('baggage_allowance_quantity').mean().alias('avg_baggage_qty'),
            pl.col('baggage_allowance_weight').mean().alias('avg_baggage_weight'),
            pl.col('max_cabin_class').mean().alias('avg_max_cabin_class'),
            pl.col('min_seats_available').mean().alias('avg_min_seats')
        ]).to_pandas().iloc[0]

        print(f"\nService Level:")
        print(f"  Cancellation Fee: {service_stats['cancellation_fee_pct']*100:.1f}%")
        print(f"  Exchange Fee: {service_stats['exchange_fee_pct']*100:.1f}%")
        print(f"  Baggage Allowance: {service_stats['avg_baggage_qty']:.1f} pieces, {service_stats['avg_baggage_weight']:.1f} kg")
        print(f"  Avg Max Cabin Class: {service_stats['avg_max_cabin_class']:.1f}")
        print(f"  Avg Min Seats Available: {service_stats['avg_min_seats']:.1f}")

        # Time of day information
        time_stats = cluster_data.select([
            pl.col('leg0_departure_hour').mean().alias('avg_leg0_dep_hour'),
            pl.col('leg0_arrival_hour').mean().alias('avg_leg0_arr_hour'),
            pl.col('leg1_departure_hour').mean().alias('avg_leg1_dep_hour'),
            pl.col('leg1_arrival_hour').mean().alias('avg_leg1_arr_hour')
        ]).to_pandas().iloc[0]

        print(f"\nTime of Day:")
        print(f"  Outbound Departure: {time_stats['avg_leg0_dep_hour']:.1f}h")
        print(f"  Outbound Arrival: {time_stats['avg_leg0_arr_hour']:.1f}h")
        if 'is_roundtrip' in cluster_data.columns and cluster_data.select(pl.col('is_roundtrip').mean()).item() > 0.5:
            print(f"  Return Departure: {time_stats['avg_leg1_dep_hour']:.1f}h")
            print(f"  Return Arrival: {time_stats['avg_leg1_arr_hour']:.1f}h")

        # Carrier and aircraft information
        if 'unique_carriers' in cluster_data.columns and 'aircraft_diversity' in cluster_data.columns:
            diversity_stats = cluster_data.select([
                pl.col('unique_carriers').mean().alias('avg_unique_carriers'),
                pl.col('aircraft_diversity').mean().alias('avg_aircraft_diversity')
            ]).to_pandas().iloc[0]

            print(f"\nDiversity:")
            print(f"  Avg Unique Carriers: {diversity_stats['avg_unique_carriers']:.1f}")
            print(f"  Avg Aircraft Types: {diversity_stats['avg_aircraft_diversity']:.1f}")

        # Price comparison features
        if 'price_rank' in cluster_data.columns:
            price_comparison = cluster_data.select([
                pl.col('price_rank').mean().alias('avg_price_rank'),
                pl.col('is_cheapest').mean().alias('cheapest_pct'),
                pl.col('is_below_median').mean().alias('below_median_pct'),
                pl.col('price_ratio_to_min').mean().alias('avg_ratio_to_min')
            ]).to_pandas().iloc[0]

            print(f"\nPrice Comparison:")
            print(f"  Avg Price Rank: {price_comparison['avg_price_rank']:.1f}")
            print(f"  Cheapest Option: {price_comparison['cheapest_pct']*100:.1f}%")
            print(f"  Below Median Price: {price_comparison['below_median_pct']*100:.1f}%")
            print(f"  Avg Price Ratio to Min: {price_comparison['avg_ratio_to_min']:.2f}x")

        # Airport hub features if available
        if 'origin_is_major_hub' in cluster_data.columns and 'destination_is_major_hub' in cluster_data.columns:
            hub_stats = cluster_data.select([
                pl.col('origin_is_major_hub').mean().alias('origin_hub_pct'),
                pl.col('destination_is_major_hub').mean().alias('dest_hub_pct'),
                pl.col('includes_major_hub').mean().alias('any_hub_pct')
            ]).to_pandas().iloc[0]

            print(f"\nMajor Hub Usage:")
            print(f"  Origin is Major Hub: {hub_stats['origin_hub_pct']*100:.1f}%")
            print(f"  Destination is Major Hub: {hub_stats['dest_hub_pct']*100:.1f}%")
            print(f"  Any Major Hub in Itinerary: {hub_stats['any_hub_pct']*100:.1f}%")

In [19]:
interpret_clusters(data, clusters, n_clusters_to_show=10)


=== Cluster Analysis ===
Total clusters: 30
Dataset size: 18,145,372 rows

CLUSTER 0 (727,536 flights, 4.0%)

Top Carriers:
  SU: 590,417 (81.2%)
  S7: 67,915 (9.3%)
  U6: 64,472 (8.9%)
  UT: 1,573 (0.2%)
  KC: 452 (0.1%)

Route Popularity: 503046.0 (avg searches per route)

Flight Characteristics:
  Avg Price: $41,857 ($36,321 median)
  Avg Duration: 377 min (6.3 hrs)
  Avg Segments: 2.2
  Roundtrip: 99.9%
  Avg Booking Lead Time: 13.4 days
  Avg Price Percentile: 80.3% (lower is cheaper)

Frequent Flyer Program Match:
  Marketing Carrier FF Match: 32.3%
  Operating Carrier FF Match: 28.2%
  Any Carrier FF Match: 32.3%

Service Level:
  Cancellation Fee: 3.2%
  Exchange Fee: 9.4%
  Baggage Allowance: 1.8 pieces, 0.0 kg
  Avg Max Cabin Class: 1.0
  Avg Min Seats Available: 4.0

Time of Day:
  Outbound Departure: 12.9h
  Outbound Arrival: 13.7h
  Return Departure: 13.8h
  Return Arrival: 14.3h

Diversity:
  Avg Unique Carriers: 1.2
  Avg Aircraft Types: 1.6

Price Comparison:
  Avg Pric

## Customer Clustering

cust_data shape: (32922, 23)


profileId,companyID,sex,nationality,frequentFlyer,isVip,bySelf,corporateTarrifCode,ff_normalized,total_searches,roundtrip_preference,unique_routes_searched,min_booking_lead_days,max_booking_lead_days,avg_booking_lead_days,median_booking_lead_days,most_common_departure_airport,unique_departure_airports,most_common_carrier,unique_carriers_used,min_cabin_class,max_cabin_class,avg_cabin_class
i64,i64,bool,i64,str,bool,bool,i64,str,u32,f64,u32,i32,i32,f64,f64,str,u32,str,u32,f64,f64,f64
3519897,60482,true,23,null,false,true,null,"""""",52,0.0,3,11,12,11.596154,12.0,"""OVB""",2,"""S7""",6,1.0,4.0,1.057692
3340584,60621,false,36,null,false,true,null,"""""",914,0.737418,2,3,8,4.78884,4.0,"""SVO""",4,"""SU""",9,1.0,2.0,1.004376
3276748,63418,true,36,null,false,true,53,"""""",16,0.0,1,8,8,8.0,8.0,"""BAX""",1,"""SU""",3,1.0,1.0,1.0
3533143,60628,true,36,null,false,true,null,"""""",14,0.0,2,1,2,1.857143,2.0,"""UUD""",2,"""S7""",3,1.0,1.0,1.0
2143289,40253,false,36,null,false,true,127,"""""",68,1.0,1,5,6,5.5,5.5,"""TOF""",1,"""S7""",1,1.0,1.0,1.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3272940,63408,true,36,null,false,true,137,"""""",18,0.0,2,6,11,8.777778,10.0,"""CEK""",2,"""SU""",2,1.0,1.0,1.0
3409276,42160,true,36,null,false,true,113,"""""",129,1.0,2,20,32,24.837209,20.0,"""TOF""",1,"""SU""",4,1.0,1.0,1.0
2886786,53375,false,36,"""SU/S7""",false,true,101,"""SU/S7""",561,0.771836,7,1,15,7.240642,9.0,"""LED""",3,"""SU""",5,1.0,2.0,1.140523


['profile_id',
 'company_id',
 'sex',
 'nationality',
 'frequent_flyer',
 'is_vip',
 'by_self',
 'corporate_tarrif_code',
 'ff_normalized',
 'total_searches',
 'roundtrip_preference',
 'unique_routes_searched',
 'min_booking_lead_days',
 'max_booking_lead_days',
 'avg_booking_lead_days',
 'median_booking_lead_days',
 'most_common_departure_airport',
 'unique_departure_airports',
 'most_common_carrier',
 'unique_carriers_used',
 'min_cabin_class',
 'max_cabin_class',
 'avg_cabin_class']

CUSTOMER FEATURE SUMMARY
Total unique customers: 32,922
Total features: 23

CATEGORICAL FEATURES:

sex:
  True: 20,713
  False: 12,209

nationality:
  26: 8
  4: 4
  45: 18
  1: 2
  33: 1

frequentFlyer:
  U6/S7/N4: 1
  SU/TK/KC: 1
  S7/N4: 1
  Y7/S7/SU/U6/N4: 1
  SU/UT/S7/U6: 5

corporateTarrifCode:
  79: 352
  55: 266
  118: 106
  150: 12
  40: 6

most_common_departure_airport:
  STW: 123
  NUX: 369
  CGK: 1
  TYN: 1
  RMZ: 158

most_common_carrier:
  CZ: 82
  Y7: 128
  6R: 4
  A3: 2
  5G: 1

NUMERICAL FEATURES:

total_searches:
  count: 32922.00
  null_count: 0.00
  mean: 551.16
  std: 1290.84
  min: 1.00
  25%: 53.00
  50%: 168.00
  75%: 505.00
  max: 28718.00

avg_booking_lead_days:
  count: 32922.00
  null_count: 0.00
  mean: 15.74
  std: 16.62
  min: 0.00
  25%: 5.00
  50%: 10.50
  75%: 20.17
  max: 164.00

unique_routes_searched:
  count: 32922.00
  null_count: 0.00
  mean: 2.23
  std: 2.06
  min: 1.00
  25%: 1.00
  50%: 2.00
  75%: 3.00
  max: 30.00

unique_departure_airports:

profile_id,company_id,sex,nationality,frequent_flyer,is_vip,by_self,corporate_tarrif_code,ff_normalized,total_searches,roundtrip_preference,unique_routes_searched,min_booking_lead_days,max_booking_lead_days,avg_booking_lead_days,median_booking_lead_days,most_common_departure_airport,unique_departure_airports,most_common_carrier,unique_carriers_used,min_cabin_class,max_cabin_class,avg_cabin_class
i64,i64,bool,i64,str,bool,bool,i64,str,u32,f64,u32,i32,i32,f64,f64,str,u32,str,u32,f64,f64,f64
3519897,60482,true,23,null,false,true,null,"""""",52,0.0,3,11,12,11.596154,12.0,"""OVB""",2,"""S7""",6,1.0,4.0,1.057692
3340584,60621,false,36,null,false,true,null,"""""",914,0.737418,2,3,8,4.78884,4.0,"""SVO""",4,"""SU""",9,1.0,2.0,1.004376
3276748,63418,true,36,null,false,true,53,"""""",16,0.0,1,8,8,8.0,8.0,"""BAX""",1,"""SU""",3,1.0,1.0,1.0
3533143,60628,true,36,null,false,true,null,"""""",14,0.0,2,1,2,1.857143,2.0,"""UUD""",2,"""S7""",3,1.0,1.0,1.0
2143289,40253,false,36,null,false,true,127,"""""",68,1.0,1,5,6,5.5,5.5,"""TOF""",1,"""S7""",1,1.0,1.0,1.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
930411,42174,true,36,"""SU/S7""",false,true,113,"""SU/S7""",62,0.33871,4,25,65,40.16129,33.0,"""BQS""",3,"""SU""",3,1.0,1.0,1.0
1447789,24728,true,36,null,false,true,161,"""""",251,1.0,1,0,1,0.47012,0.0,"""SVO""",2,"""SU""",4,1.0,1.0,1.0
2908273,61061,false,36,null,false,true,161,"""""",155,1.0,1,25,26,25.535484,26.0,"""NBC""",1,"""SU""",1,1.0,1.0,1.0


10

company_id,nationality,corporate_tarrif_code,total_searches,roundtrip_preference,unique_routes_searched,min_booking_lead_days,max_booking_lead_days,avg_booking_lead_days,median_booking_lead_days,unique_departure_airports,unique_carriers_used,min_cabin_class,max_cabin_class,avg_cabin_class,sex_encoded,frequent_flyer_encoded,is_vip_encoded,by_self_encoded,ff_normalized_encoded,most_common_departure_airport_encoded,most_common_carrier_encoded
i64,i64,i64,u32,f64,u32,i32,i32,f64,f64,u32,u32,f64,f64,f64,u32,u32,u32,u32,u32,u32,u32
60482,23,0,52,0.0,3,11,12,11.596154,12.0,2,6,1.0,4.0,1.057692,471,505,472,471,2,0,1
60621,36,0,914,0.737418,2,3,8,4.78884,4.0,4,9,1.0,2.0,1.004376,472,505,472,471,2,3,5
63418,36,53,16,0.0,1,8,8,8.0,8.0,1,3,1.0,1.0,1.0,471,505,472,471,2,4,5
60628,36,0,14,0.0,2,1,2,1.857143,2.0,2,3,1.0,1.0,1.0,471,505,472,471,2,6,1
40253,36,127,68,1.0,1,5,6,5.5,5.5,1,1,1.0,1.0,1.0,472,505,472,471,2,7,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
41022,36,59,78,1.0,1,32,32,32.0,32.0,1,2,1.0,1.0,1.0,472,505,472,471,2,53,5
61150,36,0,1197,0.848789,8,4,16,8.680869,6.0,3,11,1.0,2.0,1.046644,471,505,472,471,2,29,5
42622,36,181,73,0.0,1,0,1,0.479452,0.0,1,4,1.0,2.0,1.506849,471,505,472,471,2,24,5


array([ 7, 18, 18,  2, 29, 13,  4, 23, 23,  3,  7,  6, 25, 15, 22, 24,  7,
       25,  3,  6,  3, 26, 26, 22, 29,  3, 13, 23,  5, 29, 11, 26, 22,  3,
       15, 18,  5,  7,  3, 27, 28,  7,  6,  1,  8, 29, 13,  5,  3, 29,  8,
        8, 14,  6, 26, 22, 16,  3, 13,  4,  6,  3, 14,  8,  5,  0,  1, 28,
       26, 26,  0, 11, 24,  5, 25, 16,  4,  3,  3, 25,  1, 13, 13,  2, 16,
        6, 18,  3, 16,  6, 23, 13, 22, 24,  2, 29,  2, 26,  3,  5],
      dtype=int32)

Added cluster labels to 32922 customers
Cluster distribution:
  Cluster 0: 1,165 customers (3.5%)
  Cluster 1: 4,927 customers (15.0%)
  Cluster 2: 1,158 customers (3.5%)
  Cluster 3: 1,157 customers (3.5%)
  Cluster 4: 3,619 customers (11.0%)
  Cluster 5: 444 customers (1.3%)
  Cluster 6: 187 customers (0.6%)
  Cluster 7: 237 customers (0.7%)
  Cluster 8: 1,730 customers (5.3%)
  Cluster 9: 1,448 customers (4.4%)
  Cluster 10: 1,358 customers (4.1%)
  Cluster 11: 1,205 customers (3.7%)
  Cluster 12: 413 customers (1.3%)
  Cluster 13: 1,005 customers (3.1%)
  Cluster 14: 120 customers (0.4%)
  Cluster 15: 245 customers (0.7%)
  Cluster 16: 674 customers (2.0%)
  Cluster 17: 335 customers (1.0%)
  Cluster 18: 1,271 customers (3.9%)
  Cluster 19: 1,100 customers (3.3%)
  Cluster 20: 1,750 customers (5.3%)
  Cluster 21: 175 customers (0.5%)
  Cluster 22: 195 customers (0.6%)
  Cluster 23: 1,420 customers (4.3%)
  Cluster 24: 416 customers (1.3%)
  Cluster 25: 1,703 customers (5.2%)
  Clus

CLUSTERING QUALITY METRICS
Number of clusters: 30
Silhouette Score: 0.1125 (higher is better, range: -1 to 1)
Calinski-Harabasz Score: 1667.12 (higher is better)
Davies-Bouldin Score: 1.8170 (lower is better)

Cluster Size Statistics:
  Min cluster size: 120
  Max cluster size: 4,927
  Avg cluster size: 1097

Interpretation:
  Silhouette score indicates Poor cluster separation
Analyzing 30 clusters...
Processing Cluster 0: 1,165 customers (3.5%)
Processing Cluster 1: 4,927 customers (15.0%)
Processing Cluster 2: 1,158 customers (3.5%)
Processing Cluster 3: 1,157 customers (3.5%)
Processing Cluster 4: 3,619 customers (11.0%)
Processing Cluster 5: 444 customers (1.3%)
Processing Cluster 6: 187 customers (0.6%)
Processing Cluster 7: 237 customers (0.7%)
Processing Cluster 8: 1,730 customers (5.3%)
Processing Cluster 9: 1,448 customers (4.4%)
Processing Cluster 10: 1,358 customers (4.1%)
Processing Cluster 11: 1,205 customers (3.7%)
Processing Cluster 12: 413 customers (1.3%)
Processing Cl

## Optimize Customer Clusters

Initialized with 32,922 samples and 22 features

CURRENT CLUSTERING ANALYSIS
Metrics (calculated on 32,922 sample):
  Silhouette Score: 0.1125
  Calinski-Harabasz Score: 1667.12
  Davies-Bouldin Score: 1.8170

Cluster Balance:
  Number of clusters: 30
  Min cluster size: 120
  Max cluster size: 4,927
  Avg cluster size: 1097
  Cluster imbalance (CV): 0.933

Problematic Clusters:
  Large clusters (>1768): [ 1  4 26]
  Small clusters (<194): [ 6 14 21]

🔍 HYPERPARAMETER OPTIMIZATION
Testing 7 cluster counts × 4 batch sizes
Testing n_clusters=20, batch_size=10,000
  Avg silhouette: 0.0985 ± 0.0081
Testing n_clusters=20, batch_size=25,000
  Avg silhouette: 0.1009 ± 0.0105
Testing n_clusters=20, batch_size=50,000
  Avg silhouette: 0.1009 ± 0.0091
Testing n_clusters=20, batch_size=100,000
  Avg silhouette: 0.1009 ± 0.0091
Testing n_clusters=25, batch_size=10,000
  Avg silhouette: 0.1040 ± 0.0088
Testing n_clusters=25, batch_size=25,000
  Avg silhouette: 0.1057 ± 0.0063
Testing n_clusters=25,

### Additional Optimization Attempt

# Placehoder